<a href="https://colab.research.google.com/github/dotruni/project/blob/main/data_eda_rev00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 패키지 설치

In [ ]:
!pip3 install --upgrade torch torchvision
! set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x 
!pip install pororo
!pip install python-mecab-ko
!pip install tqdm
!pip install tqdm --upgrade
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

### 사용자 구분

In [ ]:
user = '성근'
# user = '혜람'

In [ ]:
if user == '혜람' :
    cd /content/drive/MyDrive/nlp/KnuSentiLex/data

### 패키지 import

In [ ]:
import torch
import torchvision
import torch.nn
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import datetime
import re 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
plt.style.use('seaborn-white')
from konlpy.tag import Mecab 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os 
import json
from pororo import Pororo
from konlpy.tag import Okt
from konlpy.tag import Kkma
from tqdm import tqdm
import pickle
from matplotlib import pyplot as plt
import seaborn as sns

### GPU 활성화 확인

In [ ]:
torch.cuda.is_available()

### DataFrame 불러오기

In [ ]:
if user == '성근' :
    df = pd.read_csv('M_.csv', encoding='utf-8')

elif user == '혜람' :
    pass

In [ ]:
df

### channel_url, sender_nickname columns 인코딩


In [ ]:
enc = OrdinalEncoder()
df[['channel_url']] = enc.fit_transform(df[['channel_url']])

In [ ]:
df['channel_url'] = df['channel_url'].astype("int")+1
df = df.sort_values(by=['channel_url', 'created_at']).reset_index(level=None, drop=True)

In [ ]:
pre = None
now = None
pre_date = None
name_list = []
name_dict = {}
channel_startdate = []
channel_lastdate = []
for a, b, c in zip(df['channel_url'],df['sender_nickname'],df['created_at']) :
    now = a
    if pre != now :
        count = 1
        name_dict[b] = count
        name_list.append(str(now)+"-"+str(name_dict[b]))
        channel_startdate.append((c))
        if pre_date != None :
            channel_lastdate.append(pre_date)
    else :
        if b not in name_dict :
            count += 1
            name_dict[b] = count
            name_list.append(str(now)+"-"+str(name_dict[b]))
        else :
            name_list.append(str(now)+"-"+str(name_dict[b]))
    if df['channel_url'].iloc[-1] == a and df['sender_nickname'].iloc[-1] == b and df['created_at'].iloc[-1] == c :
        channel_lastdate.append(c)
    pre = now
    pre_date = c

In [ ]:
df['sender_nickname'] = name_list

In [ ]:
df

### 시간데이터 상세 구분 및 한국시간 표기

In [ ]:
year_list = []
month_list = []
day_list = []
weekday_list = []
time_list = []
for date in df['created_at'] :
    ymd, time = date.split("+")
    ymd = datetime.datetime.strptime(ymd, "%Y-%m-%d %H:%M:%S") + datetime.timedelta(hours=9)

    year = ymd.year
    month = ymd.month
    day = ymd.day
    time = ymd.time()
    weekday = ymd.weekday()

    year_list.append(year)
    month_list.append(month)
    day_list.append(day)
    time_list.append(time)
    weekday_list.append(weekday)

In [ ]:
df['year'] = year_list
df['month'] = month_list
df['day'] = day_list
df['time'] = time_list
df['weekday'] = weekday_list

In [ ]:
df = df.drop('created_at', axis=1)

In [ ]:
df['weekday'] = df['weekday'].replace([0,1,2,3,4,5,6],['월','화','수','목','금','토','일'])

In [ ]:
df

### 시간별 구분

In [ ]:
hour_list = []
hour2_list = []
for t in df['time'] :
    hour_list.append(t.hour)
    if t.hour == 0 :
        hour2_list.append(t.hour)
    elif t.hour%2 == 0 :
        hour2_list.append(t.hour)
    elif t.hour%2 == 1 :
        hour2_list.append(t.hour-1)

In [ ]:
df['hour'] = hour_list
df['hour2'] = hour2_list

In [ ]:
df

### 시간별, 요일별, 채널별 메세지 수 시각화

In [ ]:
df_hour = df.groupby('hour').count()[['message']]
df_hour2 = df.groupby('hour2').count()[['message']]
df_weekday = df.groupby('weekday').count()[['message']]
df_channel = df.groupby('channel_url').count()[['message']]

In [ ]:
df_hour.T

In [ ]:
df_hour2.T

In [ ]:
df_weekday_rev = df_weekday.T[['월','화','수','목','금','토','일']]
df_weekday_rev

In [ ]:
sns.barplot(data=df_hour.T)
plt.show()

In [ ]:
sns.barplot(data=df_hour2.T)
plt.show()

In [ ]:
plt.rc('font', family='NanumBarunGothic')
sns.barplot(data=df_weekday_rev)
plt.show()

In [ ]:
sns.barplot(data=df_channel.T)
plt.show()

### 채널별 첫 메세지 시간, 마지막 메세지 시간, 생존기간 표기

In [ ]:
channel_startdate_rev = []
for date in channel_startdate :
    ymd, time = date.split("+")
    ymd = datetime.datetime.strptime(ymd, "%Y-%m-%d %H:%M:%S") + datetime.timedelta(hours=9)

    channel_startdate_rev.append(ymd)

channel_lastdate_rev = []
for date in channel_lastdate :
    ymd, time = date.split("+")
    ymd = datetime.datetime.strptime(ymd, "%Y-%m-%d %H:%M:%S") + datetime.timedelta(hours=9)

    channel_lastdate_rev.append(ymd)

In [ ]:
df_channel['start'] = channel_startdate_rev
df_channel['last'] = channel_lastdate_rev
df_channel['channel_survived'] = df_channel['last']-df_channel['start']

In [ ]:
df_channel.sort_values(by='channel_survived', ascending=False).head()

### 하루당 메세지 수 표기

In [ ]:
message_per_day = []
for a,b in zip(df_channel['message'],df_channel['channel_survived']) :
    time = int(str(b).split(" ")[0])
    if time == 0 :
        message_per_day.append(0)
    else :
        answer = a//time
        message_per_day.append(answer)

In [ ]:
df_channel['message_per_day'] = message_per_day
df_channel#.sort_values(by='message_per_day', ascending=False).head(50)

### 입장, 퇴장 수 카운트, 생존비율 표기



In [ ]:
pre1 = None
now1 = None
in_count = 0
out_count = 0
in_list = []
out_list = []
for a,b in zip(df['channel_url'],df['message']) :
    now1 = a
    if now1 != pre1 and pre1 != None :
        in_list.append(in_count)
        out_list.append(out_count)
        in_count = 0
        out_count = 0
    if '참여했습니다.' in b :
        in_count += 1
    if '나갔습니다.' in b :
        out_count += 1
    if a == df['channel_url'].iloc[-1] and b == df['message'].iloc[-1] :
        in_list.append(in_count)
        out_list.append(out_count)
    pre1 = now1


In [ ]:
df_channel['in_count'] = in_list
df_channel['out_count'] = out_list
df_channel['people_survived(%)'] = round((df_channel['in_count']-df_channel['out_count'])/df_channel['in_count']*100, 2)

In [ ]:
df_channel.sort_values(by=['message_per_day', 'people_survived(%)'], ascending=False)

### 데이터 저장

In [ ]:
with open("df.pickle","wb") as fw3:
    pickle.dump(df, fw3)

with open("df_channel.pickle","wb") as fw4:
    pickle.dump(df_channel, fw4)

### 메세지 정규표현식 적용(한글)

In [ ]:
df['message']=df['message'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 ]", "") #^: not임
df['message'][:30]

### 결측치 제거

In [ ]:
df_message = df
df_message['message'] = df_message['message'].replace('',np.nan)
print(len(df))
print(df.isnull().sum())

In [ ]:
df_message=df_message.dropna(how='any')
print(len(df_message))

In [ ]:
drop_index = df_message[df_message['message'].str.contains('참여했습니다|나갔습니다')].index
drop_index

try :
    df_message.drop(drop_index, inplace=True)
except :
    pass

In [ ]:
len(df_message)

### 불용어 설정

In [ ]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','거',
           '것','듯','게','게요','전','난','일','걸','잼','데','땐','ㅁ','그','나','내','그건','그걸','너','님','저'] 

### Pororo 활용한 긍정, 부정 추출 - 문장 단위

In [ ]:
sentiment = Pororo(task="sentiment", model="brainbert.base.ko.nsmc", lang="ko")
sentiment2 = Pororo(task="sentiment", model="brainbert.base.ko.shopping", lang="ko")

In [ ]:
# Pororo에서 512 이상을 지원하지 않아, 512보다 큰 문장을 나눠서 계산함.

def feel_count(message) :
    if len(message) < 512 :
        return int(sentiment(message).replace('Positive','1').replace('Negative','-1'))
    else :
        return int(feel_count(message[:len(message)//2])) + int(feel_count(message[len(message)//2:]))

def feel_count2(message) :
    if len(message) < 512 :
        return int(sentiment2(message).replace('Positive','1').replace('Negative','-1'))
    else :
        return int(feel_count(message[:len(message)//2])) + int(feel_count(message[len(message)//2:]))

In [ ]:
df_message["sentiment"]=df_message["message"].apply(lambda x: feel_count(x))
df_message["sentiment_shop"]=df_message["message"].apply(lambda x: feel_count2(x))

In [ ]:
# 나눠서 계산한 값을 긍정은 1 부정은 -1 로 계산해서 더한 후 문장 전체적인 긍정과 부정으로 나누었음

sen_list = []
for sentiment in df_message['sentiment'] :
    if sentiment > 0 :
        sen_list.append('Positive')
    else :
        sen_list.append('Negative')

sen_list2 = []
for sentiment in df_message['sentiment_shop'] :
    if sentiment > 0 :
        sen_list2.append('Positive')
    else :
        sen_list2.append('Negative')

In [ ]:
df_message['sentiment'] = sen_list
df_message['sentiment_shop'] = sen_list2
df_message

### 문장에서 단어 추출 okt 활용

In [ ]:
okt = Okt()

In [ ]:
nouns=[]
for sentence in tqdm(df_message['message']):
  nouns.append([word for word in okt.nouns(sentence) if not word in stopwords])

In [ ]:
df_message['okt'] = nouns

In [ ]:
df_message

### 단어별로 긍정 1 부정 -1로 계산하여 합산

In [ ]:
sentiment = Pororo(task="sentiment", model="brainbert.base.ko.nsmc", lang="ko")
sentiment2 = Pororo(task="sentiment", model="brainbert.base.ko.shopping", lang="ko")

In [ ]:
okt_score = []
okt_score2 = []
for okt in tqdm(df_message['okt']) :
    score = 0
    score2 = 0
    try :
        for okt2 in okt :
            score = score + int(sentiment(okt2).replace('Positive','1').replace('Negative','-1'))
            score2 = score2 + int(sentiment2(okt2).replace('Positive','1').replace('Negative','-1'))
        okt_score.append(score)
        okt_score2.append(score2)

    except :
        okt_score.append(score)
        okt_score2.append(score2)
        continue

In [ ]:
df_message['okt_score'] = okt_score
df_message['okt_score_shop'] = okt_score2
df_message

### 긍정, 부정의 평균값 확인

In [ ]:
df_message['okt_score'].sum()/len(df_message['okt_score'])

In [ ]:
df_message['okt_score_shop'].sum()/len(df_message['okt_score_shop'])

### 보기 편하게 컬럼 위치 수정

In [ ]:
df_message.columns

In [ ]:
df_message = df_message[['channel_url', 'sender_nickname', 'type', 'message', 'year', 'month',
       'day', 'time', 'weekday', 'hour', 'hour2', 'sentiment', 'sentiment_shop', 'okt',
       'okt_score', 'okt_score_shop']]

In [ ]:
df_message.iloc[550:600]

In [ ]:
with open("df_message_rev.pickle","wb") as fw:
    pickle.dump(df_message, fw)

with open("df_rev.pickle","wb") as fw2:
    pickle.dump(df, fw2)